In [4]:
import tensorflow as tf
from tensorflow import keras

In [5]:
! pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 5.0 MB/s 
     |████████████████████████████████| 1.6 MB 40.5 MB/s 


In [6]:
import keras_tuner as kt

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [8]:
mnist = tf.keras.datasets.mnist

In [9]:
(x_train, y_train), (x_test,y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


**Model without Hyperparameter Tuning**

In [10]:
model = keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(
    metrics=['accuracy'],
    optimizer='adam',
    loss='sparse_categorical_crossentropy'
)
model.fit(x_train,y_train,epochs=5)


Epoch 1/5
1875/1875 [==============================] - 7s 2ms/step - loss: 2.8235 - accuracy: 0.8761
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4416 - accuracy: 0.9059
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4052 - accuracy: 0.9114
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4052 - accuracy: 0.9124
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3631 - accuracy: 0.9221


In [11]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3194 - accuracy: 0.9470


[0.31935232877731323, 0.9470000267028809]

**With hyperparameter tuning**

In [16]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28,28)))
  hyper_parameter_units = hp.Int('units', min_value=16,max_value=512,step=16) # creating our hyperparameter tuning configuration
  model.add(keras.layers.Dense(units=hyper_parameter_units, activation='relu')) # using that configuration in a dense layer
  model.add(tf.keras.layers.Dropout(0.2)),
  model.add(keras.layers.Dense(10,activation='softmax'))

  model.compile(
    metrics=['accuracy'],
    optimizer='adam',
    loss='sparse_categorical_crossentropy' 
  )
  return model

In [17]:
# we then define our search strategy using the keras tuner library
tuner = kt.Hyperband(    
    model_builder,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='my_dir',
    project_name='intro_to_kt'
)

In [18]:
# we can set a condition for early stopping, here were monitoring the validation accuracy after 5 epochs, no significant...
# ...change means after that time frame means we skip
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)
tuner.search(x_train,y_train,epochs=50,validation_split=0.2,callbacks=[stop_early])

Trial 30 Complete [00h 00m 52s]
val_accuracy: 0.9474999904632568

Best val_accuracy So Far: 0.9474999904632568
Total elapsed time: 00h 09m 52s


In [19]:
model = keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(368, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(
    metrics=['accuracy'],
    optimizer='adam',
    loss='sparse_categorical_crossentropy'
)
model.fit(x_train,y_train,epochs=5)


Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 2.7493 - accuracy: 0.8602
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4276 - accuracy: 0.9023
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4035 - accuracy: 0.9061
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3633 - accuracy: 0.9146
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3579 - accuracy: 0.9185


In [20]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3238 - accuracy: 0.9413


[0.32377392053604126, 0.9412999749183655]